In [1]:
import os
import cv2
from deepface import DeepFace



# Process image folder

In [9]:
folder = './faces'

print(folder)

for file_name in os.listdir(folder):
    # Asume imágenes en formato png o jpg
    if file_name.endswith('.png') or file_name.endswith('.jpg'):
        # Procesa la imagen que asume hay cara, no fuerza la detección
        obj = DeepFace.analyze(img_path = os.path.join(folder, file_name), enforce_detection=False, actions =['age', 'gender', 'race', 'emotion'])
        print(file_name)
        # Acceder al primer elemento de la lista (el único elemento en este caso)
        elemento = obj[0]

        # Imprimir los valores de las claves 'region', 'age', 'gender', 'race', 'dominant_race', 'emotion', 'dominant_emotion'
        print(elemento["region"])
        print(elemento["age"])
        print(elemento["gender"])
        print(elemento["race"])
        print(elemento["dominant_race"])
        print(elemento["emotion"])
        print(elemento["dominant_emotion"])

        


./faces


Action: emotion: 100%|██████████| 4/4 [00:01<00:00,  2.38it/s]


face.jpg
{'x': 168, 'y': 206, 'w': 685, 'h': 685}
31
{'Woman': 99.96367692947388, 'Man': 0.036322182859294116}
{'asian': 99.88231062889099, 'indian': 0.04763623001053929, 'black': 2.1404525796242524e-05, 'white': 0.0020860274162259884, 'middle eastern': 4.271668530009265e-06, 'latino hispanic': 0.06794218206778169}
asian
{'angry': 5.67243162163944e-09, 'disgust': 1.2498665451463863e-17, 'fear': 4.903091127550496e-09, 'happy': 98.28951358795166, 'sad': 1.6383581158407878e-07, 'surprise': 0.0023081125618773513, 'neutral': 1.7081791535019875}
happy


Action: emotion: 100%|██████████| 4/4 [00:01<00:00,  2.73it/s]


face2.jpg
{'x': 162, 'y': 215, 'w': 690, 'h': 690}
33
{'Woman': 0.0032345276849810034, 'Man': 99.99676942825317}
{'asian': 5.3212885882458636e-05, 'indian': 9.247350226752416e-06, 'black': 2.676072155715015e-07, 'white': 99.95562434725471, 'middle eastern': 0.020072896376409813, 'latino hispanic': 0.024233049792861217}
white
{'angry': 8.862521383434796e-07, 'disgust': 1.4376012587389436e-11, 'fear': 5.715137252337854e-07, 'happy': 99.99663829803467, 'sad': 1.8166746995618155e-07, 'surprise': 1.985750452959678e-07, 'neutral': 0.003360742630320601}
happy


Action: emotion: 100%|██████████| 4/4 [00:01<00:00,  2.47it/s]

happy.jpg
{'x': 114, 'y': 92, 'w': 172, 'h': 172}
33
{'Woman': 0.7492872886359692, 'Man': 99.25071597099304}
{'asian': 0.21307447459548712, 'indian': 0.18618986941874027, 'black': 0.013073197624180466, 'white': 67.57243275642395, 'middle eastern': 9.647393226623535, 'latino hispanic': 22.36783802509308}
white
{'angry': 7.674751806296953e-09, 'disgust': 4.996208122947852e-17, 'fear': 7.288443225415397e-14, 'happy': 99.99966621398926, 'sad': 9.893928124699025e-11, 'surprise': 1.8002191751187446e-09, 'neutral': 0.0003346118774061324}
happy


In [ ]:
import cv2
import numpy as np
import FaceNormalizationUtils as faceutils
import FaceDetectors
import random
import time
import pygame

# Global variables
changing_images = False
last_image_change_time = time.time()
image_change_duration = 5  # Change the image every 10 seconds
current_image_index = 0
time_remaining = image_change_duration  # Initialize time_remaining

# Function to handle mouse click events
def on_mouse_click(event, x, y, flags, param):
    global changing_images, last_image_change_time, time_remaining
    if event == cv2.EVENT_LBUTTONDOWN:  # Left mouse button click
        changing_images = True
        last_image_change_time = time.time()
        time_remaining = image_change_duration  # Reset time_remaining

# Normalizador HS
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Inicializa la captura de la webcam
cap = cv2.VideoCapture(0)

# Variables para el manejo de la ruleta de imágenes
image_paths = ['africa.png', 'america.png', 'asian.jpg','europa.jpg',"latino.png"]  
songs_paths =[
    "Y2meta.app - 'City of Stars' (Duet ft. Ryan Gosling, Emma Stone) - La La Land Original Motion Picture Soundtrack (128 kbps).mp3",
    'you-look-lonely-i-can-fix-that-128-ytshorts.savetube.me.mp3',
    'Y2meta.app - Kavinsky - Nightcall letra Ingles_Español (128 kbps).mp3',
    "Y2meta.app - Ryan Gosling - I'm Just Ken (From Barbie The Album) [Official Audio] (128 kbps).mp3",

]

# Initialize roulette_image before entering the while loop
roulette_image = cv2.imread(image_paths[current_image_index])

pygame.mixer.init()

# Create a window to capture mouse events
cv2.namedWindow('Webcam')
cv2.setMouseCallback('Webcam', on_mouse_click)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Calculate time remaining until image change
    time_remaining = max(0, image_change_duration - (time.time() - last_image_change_time))

    # Detección de rostros en el frame de la webcam
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])
  
    if values is not None:
        face = values[0]
        x, y, w, h = face
        if w > 0 and h > 0:
            if roulette_image is not None:
                print("dentro")
                print(roulette_image)
   
                # Calculate the position for the superimposed face image above the detected face
                y_offset = max(0, y - h)  # Place it above the detected face
                x_offset = x

                             # Resize the face image to match the size of the detected face
                resized_face_image = cv2.resize(roulette_image, (w, h))

                cv2.putText(frame, f"Which Ryan Gosling are you?", (x_offset-125, y_offset-25), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 2)
                # Superpose the resized face image on the frame
                frame[y_offset:y_offset + h, x_offset:x_offset + w] = resized_face_image

    
    if changing_images:

        if current_image_index == len(image_paths):
            current_image_index = 0
        if time_remaining == 0:
            print("THE TIME HAS COME")
            print("--------------------------------------")
            print("Indice")
            print(current_image_index)
            
            print("--------------------------------------")
            changing_images = False
            last_image_change_time = time.time()
            current_image_index = random.randint(0, len(image_paths)-1)
            pygame.mixer.music.load(songs_paths[current_image_index])  # Replace with your music file
            pygame.mixer.music.play(-1)
        print("AQUI LLEGAMOS JEFE")
        print("--------------------------------------")
        print("Indice")
        print(current_image_index)
        
        print(image_paths[current_image_index])
        roulette_image = cv2.imread(image_paths[current_image_index])
        
        print(roulette_image)

        
        
        print(roulette_image)
        current_image_index += 1
        print("--------------------------------------")

            

    cv2.imshow('Webcam', frame)

    key = cv2.waitKey(1)
    if key & 0xFF == 27:  # Press Esc to exit
        # Stop the music when your program is done
        pygame.mixer.music.stop()
        break

cap.release()
cv2.destroyAllWindows()
